In [1]:
import sys
import os
sys.path.insert(0, '/home/jindal/notebooks/jindal/NER')
import fastText
import numpy as np 
from validation import compute_f1
from keras.models import Model
from keras.layers import TimeDistributed,Conv1D,Dense,Embedding,Input,Dropout,LSTM,Bidirectional,MaxPooling1D,Flatten,concatenate
from prepro import readfile,createBatches,createMatrices,iterate_minibatches,addCharInformatioin,padding
from keras.utils import plot_model,Progbar
from keras.preprocessing.sequence import pad_sequences
from keras.initializers import RandomUniform
from sklearn.metrics import precision_recall_fscore_support as score
import sklearn
import pickle, threading

from keras.utils import to_categorical
import linecache
from keras.callbacks import Callback

epochs = 50
trainable=True

/home/jindal/miniconda3/envs/NER2/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [49]:
def createMatrices(sentences, word2Idx, case2Idx, char2Idx):
    #{'numeric': 0, 'allLower': 1, 'contains_digit': 6, 'PADDING_TOKEN': 7, 'other': 4, 'allUpper': 2, 'mainly_numeric': 5, 'initialUpper': 3}
    unknownIdx = word2Idx['UNKNOWN_TOKEN']
    paddingIdx = word2Idx['PADDING_TOKEN']    
        
    dataset = []
    
    wordCount = 0
    unknownWordCount = 0
    
    for sentence in sentences:
        wordIndices = []    
        caseIndices = []
        charIndices = []
#         labelIndices = []
        
        for word,char in sentence:  
            wordCount += 1
            if word in word2Idx:
                wordIdx = word2Idx[word]
            elif word.lower() in word2Idx:
                wordIdx = word2Idx[word.lower()]                 
            else:
                wordIdx = unknownIdx
                unknownWordCount += 1
            charIdx = []
            for x in char:
                charIdx.append(char2Idx[x])
            #Get the label and map to int            
            wordIndices.append(wordIdx)
            caseIndices.append(getCasing(word, case2Idx))
            charIndices.append(charIdx)
#             labelIndices.append(label2Idx[label])
           
        dataset.append([wordIndices,caseIndices, charIndices]) 
        
    return dataset


def getCasing(word, caseLookup):   
    casing = 'other'
    
    numDigits = 0
    for char in word:
        if char.isdigit():
            numDigits += 1
            
    digitFraction = numDigits / float(len(word))
    
    if word.isdigit(): #Is a digit
        casing = 'numeric'
    elif digitFraction > 0.5:
        casing = 'mainly_numeric'
    elif word.islower(): #All lower case
        casing = 'allLower'
    elif word.isupper(): #All upper case
        casing = 'allUpper'
    elif word[0].isupper(): #is a title, initial char upper, then all lower
        casing = 'initialUpper'
    elif numDigits > 0:
        casing = 'contains_digit'
    
   
    return caseLookup[casing]

def padding(Sentences):
    maxlen = 52
    for sentence in Sentences:
        char = sentence[2]
        for x in char:
            maxlen = max(maxlen,len(x))
    for i,sentence in enumerate(Sentences):
        Sentences[i][2] = pad_sequences(Sentences[i][2],52,padding='post')
    return Sentences

In [2]:
case2Idx = {'numeric': 0, 'allLower':1, 'allUpper':2, 'initialUpper':3, 'other':4, 'mainly_numeric':5, 'contains_digit': 6, 'PADDING_TOKEN':7}
caseEmbeddings = np.identity(len(case2Idx), dtype='float32')
print(caseEmbeddings.shape)

(8, 8)


In [3]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=3

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=3


In [12]:
characters={}

with open ('/home/jindal/notebooks/jindal/NER/language_model/toy_dataset_sequences_shuffled') as f:
    for line in f:
        for word in line:
            word =str(word)
            for char in word:
#                 print(char)
                characters[char]=True

In [13]:
char2Idx={}
for char in characters:
    char2Idx[char] = len(char2Idx)

In [15]:
toy_vocab ={}

with open('toy_dataset_sequences') as f:
    for line in f:
#         print(line)
        line = line.split()
#         print(len(line))
        for word in line:
#             word = word
#             print(word)
            toy_vocab[word]=True

In [16]:
word2Idx={}
for word in toy_vocab.keys():
    word2Idx[word] = len(word2Idx)

In [17]:
vocab_size = len(word2Idx.keys())+1
print(vocab_size)

4851


In [116]:
words_input = Input(shape=(None,300), dtype='float32',name='words_input')
# words = Embedding(input_dim =50, output_dim=300, trainable=False)(words_input)
casing_input = Input(shape=(None,), dtype='int32', name='casing_input')
casing = Embedding(output_dim=caseEmbeddings.shape[1], input_dim=caseEmbeddings.shape[0], weights=[caseEmbeddings], trainable=False)(casing_input)
character_input=Input(shape=(None,52,),name='char_input')
embed_char_out=TimeDistributed(Embedding(len(char2Idx),30,embeddings_initializer=RandomUniform(minval=-0.5, maxval=0.5)), name='char_embedding')(character_input)
dropout= Dropout(0.5, name='dropout1')(embed_char_out)
conv1d_out= TimeDistributed(Conv1D(kernel_size=3, filters=30, padding='same',activation='tanh', strides=1, name='conv'))(dropout)
maxpool_out=TimeDistributed(MaxPooling1D(52), name='maxpool')(conv1d_out)
char = TimeDistributed(Flatten())(maxpool_out)
char = Dropout(0.5)(char)
output = concatenate([words_input, char])
output = Bidirectional(LSTM(200, return_sequences=False, dropout=0.50, recurrent_dropout=0.5))(output)
# output = TimeDistributed(Dense(vocab_size))(output)
# crf = CRF(len(label2Idx))
# output = crf(output)
# output = Flatten()(output)
output = Dense(vocab_size, activation='softmax')(output)
# output = Flatten()(output)
# output = Dense(vocab_size, activation='softmax')(output)
# model.add(Dense(vocab_size, activation='softmax'))
model = Model(inputs=[words_input,casing_input, character_input], outputs=[output])
# model.compile(loss='sparse_categorical_crossentropy', optimizer='nadam', metrics=['accuracy'])
# model.summary()

ValueError: A `Concatenate` layer requires inputs with matching shapes except for the concat axis. Got inputs shapes: [(None, None), (None, None, 30)]

In [19]:
model.load_weights('german_lm.h5')

In [20]:
for layer in model.layers:
    print(layer.name)

char_input
char_embedding
dropout1
time_distributed_3
maxpool
time_distributed_4
words_input
dropout_2
concatenate_2
bidirectional_2
dense_1


In [21]:
model.layers.pop()

In [22]:
for layer in model.layers:
    print(layer.name)

char_input
char_embedding
dropout1
time_distributed_3
maxpool
time_distributed_4
words_input
dropout_2
concatenate_2
bidirectional_2


In [23]:
output = Dense(1, activation = 'softmax')(model.layers[-1].output)

In [25]:
final_model = Model(input=model.input, output=[output])

/home/jindal/miniconda3/envs/NER2/lib/python3.5/site-packages/ipykernel_launcher.py:1: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=[<tf.Tenso..., inputs=[<tf.Tenso...)`
  """Entry point for launching an IPython kernel.


In [26]:
for layer in final_model.layers:
    print(layer.name)

char_input
char_embedding
dropout1
time_distributed_3
maxpool
time_distributed_4
words_input
dropout_2
concatenate_2
bidirectional_2
dense_2


In [27]:
final_model.compile(loss='categorical_crossentropy', optimizer='nadam', metrics=['accuracy'])
final_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
char_input (InputLayer)         (None, None, 52)     0                                            
__________________________________________________________________________________________________
char_embedding (TimeDistributed (None, None, 52, 30) 3300        char_input[0][0]                 
__________________________________________________________________________________________________
dropout1 (Dropout)              (None, None, 52, 30) 0           char_embedding[0][0]             
__________________________________________________________________________________________________
time_distributed_3 (TimeDistrib (None, None, 52, 30) 2730        dropout1[0][0]                   
__________________________________________________________________________________________________
maxpool (T

In [45]:
out_file1 = open('/home/jindal/notebooks/jindal/NER/language_model/germeval2018.training_simple_labels.txt','wb')
out_file2 = open('/home/jindal/notebooks/jindal/NER/language_model/germeval2018.training_complex_labels.txt','wb')
with open('/home/jindal/notebooks/jindal/NER/language_model/germeval2018.training.txt', ) as f:
    for line in f:
        line = line.split('\t')
#         print(len(line))
        text, label_simple, label_complex = line
        simple = [text, label_simple]
        compl = [text, label_complex]
        string_simple = '\t'.join(simple)+' \n'
        string_complex = '\t'.join(compl)+' \n'
        out_file1.write(string_simple.encode())
        out_file2.write(string_complex.encode())
out_file1.close()
out_file2.close()

In [34]:
line_number=0

In [29]:
ft = fastText.load_model("/home/jindal/notebooks/fastText/wiki.de.bin")

nb_embedding_dims = ft.get_dimension()

In [42]:
characters={}
labels = {}
with open('/home/jindal/notebooks/jindal/NER/language_model/germeval2018.training_simple_labels.txt') as f:
    for line in f:
        text, label = line.split('\t')
        labels[label]=True
        for word in text:
            for char in word:
                characters[char]=True

In [43]:
print(labels)

{'OTHER \n': True, 'OFFENSE \n': True}


In [46]:
label2Idx = {}
for label in labels:
    label2Idx[label] = len(labels)

In [37]:
char2Idx={}
for char in characters:
    char2Idx[char] = len(char2Idx)

In [31]:
def file_len(fname):
    with open(fname) as f:
        for i, l in enumerate(f):
            pass
    return i + 1

In [102]:
linecache.getline('/home/jindal/notebooks/jindal/NER/language_model/germeval2018.training_simple_labels.txt', 64)

'@Miquwarchar Hat die jemand dafür angezeigt?\tOTHER \n'

In [122]:
def my_generator(file :"input training file", batch_size):
    
    global line_number, lock

    while True:
        word_embeddings = []
        case_embeddings = []
        char_embeddings = []
        output_labels = []
        for i in range(batch_size):
            
#             lock.acquire()
            total_lines = file_len(file)
            index = line_number%total_lines +1
            line_number+=1
#             lock.release()
#             index = random.choice(len(features), 1)[0]
            line = linecache.getline(file, index)
            if len(line.split('\t'))!=2:
                continue
            text, label = line.split('\t')
            assert index<=100
#             print(text)
            temp_casing = []
            temp_char=[]
            temp_word=[]
#             print(line)
            print(len)
            for word in text.split():
#                 print(word)
                casing =getCasing(word, case2Idx)
#                 print(casing)
                temp_casing.append(casing)
                temp_char2=[]
                for char in word:
                    temp_char2.append(char2Idx[char])
                temp_char2 = np.array(temp_char2)
#                 print(temp_char2)
#                 temp_char2 = pad_sequences(temp_char2, 52, padding='post')
#                 temp_char.append(pad_sequences(temp_char2, 52, padding='post'))
                temp_char.append(temp_char2)
                word_vector = ft.get_word_vector(word.lower())
#                 print(len(word_vector))
                temp_word.append(word_vector)
            temp_char = pad_sequences(temp_char, 52)
#             print(temp_word)
#             print(len(temp_word))
#             print(len(temp_casing))
#             print(temp_char)
#             print(len(temp_char))
#             print(label2Idx[label])
            print(index)
            print(" **************** ")
            word_embeddings.append(temp_word)
#             print((len(word_embeddings), len(word_embeddings[0]), len(word_embeddings[0][0])))

            case_embeddings.append(temp_casing)
            char_embeddings.append(temp_char)
            output_labels.append(label2Idx[label])
#             print(np.array(word_embeddings))
#             batch_features[i] = process_features(line, window_size-1, nb_embedding_dims)
            # print(batch_features[i])
            # print(batch_features[i].shape)
#             batch_labels[i] = labels[index]
        yield ([np.array(word_embeddings), np.array(case_embeddings), np.array(char_embeddings)], np.array(output_labels))

In [114]:
line_number=0
for inp, output in my_generator('/home/jindal/notebooks/jindal/NER/language_model/germeval2018.training_simple_labels.txt',32):
    continue
#     print(inp[0].shape)
#     print(inp[1].shape)
#     print(inp[2].shape)
#     print(output.shape)

15
1
 **************** 
(1, 15, 300)
19
2
 **************** 
(2, 15, 300)
11
3
 **************** 
(3, 15, 300)
19
4
 **************** 
(4, 15, 300)
16
5
 **************** 
(5, 15, 300)
46
6
 **************** 
(6, 15, 300)
34
7
 **************** 
(7, 15, 300)
8
8
 **************** 
(8, 15, 300)
12
9
 **************** 
(9, 15, 300)
15
10
 **************** 
(10, 15, 300)
14
11
 **************** 
(11, 15, 300)
26
12
 **************** 
(12, 15, 300)
39
13
 **************** 
(13, 15, 300)
22
14
 **************** 
(14, 15, 300)
43
15
 **************** 
(15, 15, 300)
26
16
 **************** 
(16, 15, 300)
19
17
 **************** 
(17, 15, 300)
22
18
 **************** 
(18, 15, 300)
24
19
 **************** 
(19, 15, 300)
14
20
 **************** 
(20, 15, 300)
21
21
 **************** 
(21, 15, 300)
19
22
 **************** 
(22, 15, 300)
8
23
 **************** 
(23, 15, 300)
17
24
 **************** 
(24, 15, 300)
31
25
 **************** 
(25, 15, 300)
17
26
 **************** 
(26, 15, 300)
15
27


AssertionError: 

In [123]:
line_number=0
training_file_name = '/home/jindal/notebooks/jindal/NER/language_model/germeval2018.training_simple_labels.txt'
final_model.fit_generator(my_generator(training_file_name, 1)
                         , epochs =1 , steps_per_epoch = file_len(training_file_name)//1)

Epoch 1/1
<built-in function len>
1
 **************** 
<built-in function len>
2
 **************** 


ValueError: You are passing a target array of shape (1, 1) while using as loss `categorical_crossentropy`. `categorical_crossentropy` expects targets to be binary matrices (1s and 0s) of shape (samples, classes). If your targets are integer classes, you can convert them to the expected format via:
```
from keras.utils import to_categorical
y_binary = to_categorical(y_int)
```

Alternatively, you can use the loss function `sparse_categorical_crossentropy` instead, which does expect integer targets.